# Jscatter: Lattices

This Notebook aims to give a short introduction about usage of lattices also for 2D fitting of scattering images.

### Content
- Ewald sphere of a simple cubic lattice
- 2D Fitting: 2D pattern of a lattice

![Jscatter](https://jscatter.readthedocs.io/en/latest/_images/Jscatter.jpeg "Jscatter")

#### Prerequisite 

In [ ]:
import sys
!{sys.executable} -m pip install jscatter
%matplotlib notebook

import jscatter as js
import numpy as np
js.usempl(True)   # force matplotlib, not needed in Linux

## Ewald sphere of a simple cubic lattice

In [ ]:
import jscatter as js
import numpy as np

import matplotlib.pyplot as pyplot
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D

phi, theta = np.meshgrid(np.r_[0:np.pi:45j], np.r_[0:2 * np.pi:90j])

# The Cartesian coordinates of the Ewald sphere
q = 3
x = q * (np.sin(phi) * np.cos(theta) + 1)
y = q * np.sin(phi) * np.sin(theta)
z = q * np.cos(phi)
qxzy = np.asarray([x, y, z]).reshape(3, -1).T

### Create basic lattice 

In [ ]:
sclattice = js.lattice.scLattice(2.1, 5)
sclattice.show()

### Calculate the scattering pattern of an oriented sclattice and plot the corresponding Ewald sphere

In [ ]:
ffe = js.ff.orientedCloudScattering(qxzy, sclattice.XYZ, coneangle=5, nCone=20, rms=0.02)

# log scale for colors
ffey = np.log(ffe.Y)
fmax, fmin = ffey.max(), ffey.min()
ffeY = (np.reshape(ffey, x.shape) - fmin) / (fmax - fmin)

# Set the aspect ratio to 1 so our sphere looks spherical
fig = pyplot.figure(figsize=pyplot.figaspect(1.))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=cm.seismic(ffeY))
fig.suptitle('Ewald sphere of simple cubic lattice')

# Turn off the axis planes
ax.set_axis_off()
pyplot.show(block=False)

## Show Ewald sphere and scattering plane (at detector) of a  simple cubic lattice

In [ ]:
phi, theta = np.meshgrid(np.r_[0:np.pi:90j], np.r_[0:1 * np.pi:90j])
# The Cartesian coordinates of the unit sphere
q = 8
x = q * (np.sin(phi) * np.cos(theta) + 1)
y = q * np.sin(phi) * np.sin(theta)
z = q * np.cos(phi)
qxzy = np.asarray([x, y, z]).reshape(3, -1).T

# Set the aspect ratio to 1 so our sphere looks spherical
fig = pyplot.figure(figsize=pyplot.figaspect(1.))
ax = fig.add_subplot(111, projection='3d')

# create lattice and show it scaled
sclattice = js.lattice.scLattice(2.1, 1)
sclattice.rotatehkl2Vector([1, 1, 1], [0, 0, 1])
gg = ax.scatter(sclattice.X / 3 + q, sclattice.Y / 3, sclattice.Z / 3, c='k', alpha=0.9)
gg.set_visible(True)

ds = 15
fpi = np.pi / 180.
ffs = js.sf.orientedLatticeStructureFactor(qxzy, sclattice, rotation=None, domainsize=ds, rmsd=0.1, hklmax=2, nGauss=23)
# show scattering in log scale on Ewald sphere
ffsy = np.log(ffs.Y)
fmax, fmin = ffsy.max(), ffsy.min()
ffsY = (np.reshape(ffsy, x.shape) - fmin) / (fmax - fmin)

ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=cm.hsv(ffsY), alpha=0.4)
ax.plot([q, 2 * q], [0, 0], [0, 0], color='k')
pyplot.show(block=False)

#  ad flat detector xy plane
xzw = np.mgrid[-8:8:80j, -8:8:80j]
qxzw = np.stack([np.zeros_like(xzw[0]), xzw[0], xzw[1]], axis=0)

ff2 = js.sf.orientedLatticeStructureFactor(qxzw.reshape(3, -1).T, sclattice, rotation=None, domainsize=ds, rmsd=0.1,
                                           hklmax=2, nGauss=23)
ffs2 = np.log(ff2.Y)
fmax, fmin = ffs2.max(), ffs2.min()
ff2Y = (np.reshape(ffs2, xzw[0].shape) - fmin) / (fmax - fmin)
ax.plot_surface(qxzw[0], qxzw[1], qxzw[2], rstride=1, cstride=1, facecolors=cm.gist_ncar(ff2Y), alpha=0.3)
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')

fig.suptitle('Scattering plane and Ewald sphere of lattice \n matplotlib is limited in 3D')
pyplot.show(block=False)
# matplotlib is not real 3D rendering
# maybe use later something else

## 2D Fitting: 2D pattern of a lattice
The loaded image is similar to SAXS images from ordered particles.

Looking in 3D along xy lane one can see at high q the plateau due to diffusive scattering which vanishes at low q. 

In [ ]:
import jscatter as js
import numpy as np

# load a image with hexagonal peaks (synthetic data) and set center
image = js.sas.sasImage(js.examples.datapath + '/2Dhex.tiff')
image.setPlaneCenter([51, 51])
# transform to dataarray with X,Z as wavevectors
# The fit algorithm works also with masked areas
hexa = image.asdataArray(masked=0)

fig = js.mpl.surface(hexa.X, hexa.Z, hexa.Y,shape=image.shape)
fig.suptitle('original')
fig.show()

### Model

In [ ]:
def latticemodel(qx, qz, R, ds, rmsd, hklmax, bgr, I0):
    # a hexagonal grid with background, domain size and Debye Waller-factor (rmsd)
    # 3D wavevector
    qxyz = np.c_[qx, qz, np.zeros_like(qx)]
    # define lattice
    grid = js.sf.hcpLattice(R, [3, 3, 3])
    # here one may rotate the crystal

    # calc scattering
    lattice = js.sf.orientedLatticeStructureFactor(qxyz, grid, domainsize=ds, rmsd=rmsd, hklmax=hklmax)
    # add I0 and background
    lattice.Y = I0 * lattice.Y + bgr
    return lattice

### First *differential evolution* then polish
Because of the high plateau in the chi2 landscape we first need to use a algorithm finding a global minimum. The 'differential evolution'needs around 2300 evaluations and some time.

Alternatively one may find in other ways good starting parameters. in this case one should first evaluate in 1D the peak positions and fit later the orientation. As a third step one can fit all together. 


In [ ]:
if 0:
    # Please do this only if you have time to wait
    hexa.fit(latticemodel, {'R': 2, 'ds': 10, 'rmsd': 0.1, 'bgr': 1, 'I0': 10},
             {'hklmax': 4, }, {'qx': 'X', 'qz': 'Z'}, method='differential_evolution')
    #
    fig = js.mpl.surface(hexa.X, hexa.Z, hexa.Y, shape=image.shape)
    fig = js.mpl.surface(hexa.lastfit.X, hexa.lastfit.Z, hexa.lastfit.Y)
    
# We use as starting parameters the result of the previous fit
# We use LevenbergMarquardt algorithm to polish the result
hexa.fit(latticemodel, {'R': 3.02, 'ds': 8.7, 'rmsd': 0.193, 'bgr': 3, 'I0': 31},
         {'hklmax': 4, }, {'qx': 'X', 'qz': 'Z'},output=0)


fig2 = js.mpl.surface(hexa.lastfit.X, hexa.lastfit.Z, hexa.lastfit.Y ,shape=image.shape)
fig2.suptitle('fit result')
fig2.show()

## For more examples please visit the Jscatter documentation.